In [1]:
import os
import pickle 
import numpy as np
import pandas as pd
import re
from pprint import pprint

In [2]:
output_dir="../../data/train_pid_paragraph_pos"
start="train_paragraph_pos_"
end=".pkl"
pkl_files = [fname for fname in os.listdir(output_dir) if fname.startswith(start) and fname.endswith(end)]
def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]
pkl_files.sort(key=natural_key)
print(pkl_files)


['train_paragraph_pos_0.pkl', 'train_paragraph_pos_1.pkl', 'train_paragraph_pos_2.pkl', 'train_paragraph_pos_3.pkl', 'train_paragraph_pos_4.pkl', 'train_paragraph_pos_5.pkl']


### 파일 크기

In [3]:
for file_path in sorted(pkl_files):
    size = os.path.getsize(os.path.join(output_dir, file_path))
    print(f"파일 크기: {size} 바이트")

파일 크기: 292505716 바이트
파일 크기: 294683134 바이트
파일 크기: 294973189 바이트
파일 크기: 297629285 바이트
파일 크기: 296104158 바이트
파일 크기: 303195295 바이트


### 저장 형태 확인

In [4]:
with open(os.path.join(output_dir, pkl_files[0]), "rb") as f:
    train_paragraph_pos_0=pickle.load(f)


In [5]:
train_paragraph_pos_0.head(1)

,PID,title,paragraph_index,paragraph_pos,generated
0,0,카호올라웨섬,0,"[(카호올라웨, NNP), (섬, NNG), (은, JX), (하와이, NNP), ...",0


In [6]:
train_paragraph_pos_0.dtypes

PID                 int64
title              object
paragraph_index     int64
paragraph_pos      object
generated           int64
dtype: object

### 전체 데이터 개수 및 title 연속 여부

In [7]:
data_points=[]
titles=[]
paragraph=[]

nan_list=[]
nan_details = []
df_null = None
for file_name in pkl_files:
    with open(os.path.join(output_dir, file_name), "rb") as f:
        sentence=pickle.load(f)
        # data points
        titles.append(sentence["title"].nunique())
        paragraph.append(sentence.groupby(['title', 'paragraph_index']).ngroups)
        data_points.append(len(sentence))
        # title 연속 여부
        block_titles = sentence['title'][sentence['title'] != sentence['title'].shift()].tolist()
        unique_titles = sentence['title'].unique().tolist()
        if sorted(block_titles) == sorted(unique_titles):
            print(f"{file_name}: ✅ 모든 title 연속 ⭕️")
        else:
            print(f"{file_name}⚠️  모든 title 연속 ❌")
        # 결측치
        nan_bool=sentence.isnull().values.any()
        nan_list.append(nan_bool)
        if nan_bool:
            df_null = sentence[sentence.isnull().any(axis=1)]
            nan_details.append((file_name, df_null))

train_paragraph_pos_0.pkl: ✅ 모든 title 연속 ⭕️
train_paragraph_pos_1.pkl: ✅ 모든 title 연속 ⭕️
train_paragraph_pos_2.pkl: ✅ 모든 title 연속 ⭕️
train_paragraph_pos_3.pkl: ✅ 모든 title 연속 ⭕️
train_paragraph_pos_4.pkl: ✅ 모든 title 연속 ⭕️
train_paragraph_pos_5.pkl: ✅ 모든 title 연속 ⭕️


In [8]:
print(f"title: {sum(titles)}")
print(f"paragraph: {sum(paragraph)}")
print(f"total(sentence): {sum(data_points)}")

title: 97172
paragraph: 1226364
total(sentence): 1226364


### 결측치

In [9]:
if not nan_list:
    print("⚠️ nan_list ❌")
elif any(nan_list):
    print("⚠️ 결측치 존재")
else:
    print("✅ 결측치 없음")


✅ 결측치 없음
